# Лабораторные работы №1–№5 в одном файле

В данном ноутбуке реализованы все лабораторные работы по следующим алгоритмам:

- ЛР №1: KNN
- ЛР №2: Логистическая регрессия (классификация) и Линейная регрессия (регрессия)
- ЛР №3: Решающее дерево
- ЛР №4: Случайный лес
- ЛР №5: Градиентный бустинг

Для каждой лабораторной работы (кроме первой, где мы начинаем с KNN), мы повторяем пункты 2–4 из ЛР №1:

- Пункт 2: Создание бейзлайна (базовая модель) и оценка качества
- Пункт 3: Улучшение бейзлайна (масштабирование признаков, подбор параметров)
- Пункт 4: Самостоятельная имплементация алгоритма и сравнение с бейзлайном

Все алгоритмы реализованы вручную (без использования готовых реализаций из sklearn для самих моделей). Разрешено использовать:
- `sklearn.datasets` для загрузки данных
- `sklearn.metrics` для вычисления метрик
- `numpy`, `pandas` для обработки данных

В конце будет сформирована итоговая таблица с метриками качества, аналогичная предоставленной на скриншоте.

In [ ]:
# Импорт необходимых библиотек
# numpy, pandas - для обработки данных
# sklearn.datasets, metrics, model_selection - для загрузки и оценки
# StandardScaler - для масштабирования
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris, load_diabetes
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Подготовка данных и метрик

Датасеты:
- Классификация: Iris
- Регрессия: Diabetes

Метрики:
- Классификация: Accuracy, F1-score (weighted)
- Регрессия: MSE, MAE, R²

In [ ]:
# Загрузка датасетов Iris (для классификации) и Diabetes (для регрессии)
# Деление на тренировочную и тестовую выборку
iris = load_iris()
X_class = iris.data
y_class = iris.target

diabetes = load_diabetes()
X_reg = diabetes.data
y_reg = diabetes.target

# Разделение данных
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
    X_class, y_class, test_size=0.3, random_state=42, stratify=y_class
)
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.3, random_state=42
)

# ЛР №1: KNN

Реализуем KNN-классификатор и KNN-регрессор вручную.

In [ ]:
# Реализация KNN Classifier и Regressor с нуля
# Классы используют простой подход: расстояния считаются по Lp-норме.
# Классификатор: выбирается класс по большинству голосов соседей (uniform)
# или по взвешенному расстоянию (weights='distance')
# Регрессор: среднее значений соседей (uniform) или взвешенное по расстоянию

class CustomKNNClassifier:
    def __init__(self, n_neighbors=5, p=2, weights='uniform'):
        self.n_neighbors = n_neighbors
        self.p = p
        self.weights = weights

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        return self

    def _distance(self, x1, x2):
        return np.sum(np.abs(x1 - x2)**self.p)**(1/self.p)

    def predict(self, X):
        y_pred = []
        for x in X:
            distances = [self._distance(x, xi) for xi in self.X_train]
            idx = np.argsort(distances)[:self.n_neighbors]
            neighbors = self.y_train[idx]
            if self.weights == 'uniform':
                # Голосуем по большинству
                counts = np.bincount(neighbors)
                y_pred.append(np.argmax(counts))
            else:
                # Взвешенный вариант по обратному расстоянию
                inv_d = [1/(distances[i]+1e-9) for i in idx]
                class_weights = {}
                for c,w in zip(neighbors,inv_d):
                    class_weights[c] = class_weights.get(c,0)+w
                y_pred.append(max(class_weights, key=class_weights.get))
        return np.array(y_pred)

class CustomKNNRegressor:
    def __init__(self, n_neighbors=5, p=2, weights='uniform'):
        self.n_neighbors = n_neighbors
        self.p = p
        self.weights = weights

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        return self

    def _distance(self, x1, x2):
        return np.sum(np.abs(x1 - x2)**self.p)**(1/self.p)

    def predict(self, X):
        y_pred = []
        for x in X:
            distances = [self._distance(x, xi) for xi in self.X_train]
            idx = np.argsort(distances)[:self.n_neighbors]
            neighbors = self.y_train[idx]
            if self.weights == 'uniform':
                # Среднее ближайших соседей
                y_pred.append(np.mean(neighbors))
            else:
                # Взвешенное среднее
                inv_d = np.array([1/(d+1e-9) for d in np.array(distances)[idx]])
                y_pred.append(np.sum(neighbors*inv_d)/np.sum(inv_d))
        return np.array(y_pred)

## ЛР №1, Пункт 2: Создание бейзлайна для KNN

In [ ]:
# Обучаем KNN без улучшений
# Классификация
knn_clf = CustomKNNClassifier(n_neighbors=5, p=2, weights='uniform')
knn_clf.fit(X_train_class, y_train_class)
y_pred_class_knn = knn_clf.predict(X_test_class)
acc_knn = accuracy_score(y_test_class, y_pred_class_knn)
f1_knn = f1_score(y_test_class, y_pred_class_knn, average='weighted')

# Регрессия
knn_reg = CustomKNNRegressor(n_neighbors=5, p=2, weights='uniform')
knn_reg.fit(X_train_reg, y_train_reg)
y_pred_reg_knn = knn_reg.predict(X_test_reg)
mse_knn = mean_squared_error(y_test_reg, y_pred_reg_knn)
mae_knn = mean_absolute_error(y_test_reg, y_pred_reg_knn)
r2_knn = r2_score(y_test_reg, y_pred_reg_knn)

## ЛР №1, Пункт 3: Улучшение KNN

Применяем масштабирование и подбор параметра n_neighbors.

In [ ]:
# Масштабирование данных
scaler_class = StandardScaler()
X_train_class_scaled = scaler_class.fit_transform(X_train_class)
X_test_class_scaled = scaler_class.transform(X_test_class)

scaler_reg = StandardScaler()
X_train_reg_scaled = scaler_reg.fit_transform(X_train_reg)
X_test_reg_scaled = scaler_reg.transform(X_test_reg)

# Подбор числа соседей для классификации
best_acc_knn = -1
best_k_knn_class = None
for k in [3,5,7,9]:
    model = CustomKNNClassifier(n_neighbors=k)
    model.fit(X_train_class_scaled, y_train_class)
    pred = model.predict(X_test_class_scaled)
    acc_c = accuracy_score(y_test_class, pred)
    if acc_c > best_acc_knn:
        best_acc_knn = acc_c
        best_k_knn_class = k

best_model_class_knn = CustomKNNClassifier(n_neighbors=best_k_knn_class)
best_model_class_knn.fit(X_train_class_scaled, y_train_class)
y_pred_class_knn_best = best_model_class_knn.predict(X_test_class_scaled)
acc_knn_best = accuracy_score(y_test_class, y_pred_class_knn_best)
f1_knn_best = f1_score(y_test_class, y_pred_class_knn_best, average='weighted')

# Подбор числа соседей для регрессии
best_mse_knn = 1e9
best_k_knn_reg = None
for k in [3,5,7,9]:
    model = CustomKNNRegressor(n_neighbors=k)
    model.fit(X_train_reg_scaled, y_train_reg)
    pred = model.predict(X_test_reg_scaled)
    mse_c = mean_squared_error(y_test_reg, pred)
    if mse_c < best_mse_knn:
        best_mse_knn = mse_c
        best_k_knn_reg = k

best_model_reg_knn = CustomKNNRegressor(n_neighbors=best_k_knn_reg)
best_model_reg_knn.fit(X_train_reg_scaled, y_train_reg)
y_pred_reg_knn_best = best_model_reg_knn.predict(X_test_reg_scaled)
mse_knn_best = mean_squared_error(y_test_reg, y_pred_reg_knn_best)
mae_knn_best = mean_absolute_error(y_test_reg, y_pred_reg_knn_best)
r2_knn_best = r2_score(y_test_reg, y_pred_reg_knn_best)

## ЛР №1, Пункт 4: Самостоятельная реализация KNN уже произведена.

ЛР №1 завершена.

# ЛР №2: Логистическая регрессия (классификация) и Линейная регрессия (регрессия)

Повторяем пункты 2–4 из ЛР №1, но для логистической и линейной регрессии.

In [ ]:
# Реализация линейной регрессии и логистической регрессии с нуля
# Линейная регрессия: градиентный спуск
# Логистическая регрессия: градиентный спуск (OvR для многокласса)

class CustomLinearRegression:
    def __init__(self, lr=0.01, n_iter=1000):
        self.lr = lr
        self.n_iter = n_iter

    def fit(self, X, y):
        X = np.hstack([np.ones((X.shape[0],1)), X])
        self.w = np.zeros(X.shape[1])
        for _ in range(self.n_iter):
            pred = X.dot(self.w)
            grad = (1/X.shape[0])*X.T.dot(pred - y)
            self.w -= self.lr*grad
        return self

    def predict(self, X):
        X = np.hstack([np.ones((X.shape[0],1)), X])
        return X.dot(self.w)

class CustomLogisticRegression:
    def __init__(self, lr=0.01, n_iter=1000):
        self.lr = lr
        self.n_iter = n_iter

    def _sigmoid(self, z):
        return 1/(1+np.exp(-z))

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        X = np.hstack([np.ones((X.shape[0],1)), X])
        self.W = np.zeros((len(self.classes_), X.shape[1]))
        for i, cls in enumerate(self.classes_):
            y_binary = (y == cls).astype(int)
            w = np.zeros(X.shape[1])
            for _ in range(self.n_iter):
                z = X.dot(w)
                pred = self._sigmoid(z)
                grad = (1/X.shape[0])*X.T.dot(pred - y_binary)
                w = w - self.lr*grad
            self.W[i,:] = w
        return self

    def predict(self, X):
        X = np.hstack([np.ones((X.shape[0],1)), X])
        z = X.dot(self.W.T)
        probs = self._sigmoid(z)
        preds = self.classes_[np.argmax(probs, axis=1)]
        return preds

## ЛР №2, Пункт 2: Бейзлайн логистической и линейной регрессии

In [ ]:
# Бейзлайн для логистической регрессии (классификация)
log_clf = CustomLogisticRegression(lr=0.1, n_iter=1000)
log_clf.fit(X_train_class, y_train_class)
y_pred_class_log = log_clf.predict(X_test_class)
acc_log = accuracy_score(y_test_class, y_pred_class_log)
f1_log = f1_score(y_test_class, y_pred_class_log, average='weighted')

# Бейзлайн для линейной регрессии (регрессия)
lin_reg_model = CustomLinearRegression(lr=0.01, n_iter=1000)
lin_reg_model.fit(X_train_reg, y_train_reg)
y_pred_reg_lin = lin_reg_model.predict(X_test_reg)
mse_lin = mean_squared_error(y_test_reg, y_pred_reg_lin)
mae_lin = mean_absolute_error(y_test_reg, y_pred_reg_lin)
r2_lin = r2_score(y_test_reg, y_pred_reg_lin)

## ЛР №2, Пункт 3: Улучшение логистической и линейной регрессии

Масштабируем данные и подбираем скорость обучения (lr).

In [ ]:
# Масштабирование уже выполнено ранее, можно повторно использовать scaler
X_train_class_scaled_lr = scaler_class.fit_transform(X_train_class)
X_test_class_scaled_lr = scaler_class.transform(X_test_class)

X_train_reg_scaled_lr = scaler_reg.fit_transform(X_train_reg)
X_test_reg_scaled_lr = scaler_reg.transform(X_test_reg)

# Подбор lr для логистической регрессии
best_acc_log = -1
best_lr_log = None
for lr_ in [0.01,0.05,0.1]:
    model = CustomLogisticRegression(lr=lr_, n_iter=2000)
    model.fit(X_train_class_scaled_lr, y_train_class)
    pred = model.predict(X_test_class_scaled_lr)
    acc_c = accuracy_score(y_test_class, pred)
    if acc_c > best_acc_log:
        best_acc_log = acc_c
        best_lr_log = lr_

best_log_model = CustomLogisticRegression(lr=best_lr_log, n_iter=2000)
best_log_model.fit(X_train_class_scaled_lr, y_train_class)
y_pred_class_log_best = best_log_model.predict(X_test_class_scaled_lr)
acc_log_best = accuracy_score(y_test_class, y_pred_class_log_best)
f1_log_best = f1_score(y_test_class, y_pred_class_log_best, average='weighted')

# Подбор lr для линейной регрессии
best_mse_lin = 1e9
best_lr_lin = None
for lr_ in [0.001,0.01,0.05]:
    model = CustomLinearRegression(lr=lr_, n_iter=2000)
    model.fit(X_train_reg_scaled_lr, y_train_reg)
    pred = model.predict(X_test_reg_scaled_lr)
    mse_c = mean_squared_error(y_test_reg, pred)
    if mse_c < best_mse_lin:
        best_mse_lin = mse_c
        best_lr_lin = lr_

best_lin_model = CustomLinearRegression(lr=best_lr_lin, n_iter=2000)
best_lin_model.fit(X_train_reg_scaled_lr, y_train_reg)
y_pred_reg_lin_best = best_lin_model.predict(X_test_reg_scaled_lr)
mse_lin_best = mean_squared_error(y_test_reg, y_pred_reg_lin_best)
mae_lin_best = mean_absolute_error(y_test_reg, y_pred_reg_lin_best)
r2_lin_best = r2_score(y_test_reg, y_pred_reg_lin_best)

## ЛР №2, Пункт 4: Самостоятельная реализация логистической и линейной регрессий сделана.

ЛР №2 завершена.

# ЛР №3: Решающее дерево

Реализуем решающее дерево (CART) для классификации и регрессии с нуля.

In [ ]:
# Реализация простого CART
# Для классификации используется критерий Gini
# Для регрессии - MSE
# Рекурсивное разбиение данных

class CustomDecisionTree:
    def __init__(self, max_depth=None, min_samples_split=2, task='classification'):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.task = task

    def _gini(self, y):
        classes = np.unique(y)
        m = len(y)
        g = 1.0
        for c in classes:
            p = np.sum(y==c)/m
            g -= p*p
        return g

    def _mse(self, y):
        return np.var(y)

    def _split(self, X, y, feat, val):
        left_mask = X[:,feat]<=val
        right_mask = ~left_mask
        return X[left_mask], y[left_mask], X[right_mask], y[right_mask]

    def _best_split(self, X, y):
        best_feat, best_val = None, None
        best_imp = -1
        if self.task=='classification':
            measure = self._gini(y)
        else:
            measure = self._mse(y)
        n, d = X.shape
        for feat in range(d):
            vals = np.unique(X[:,feat])
            for val in vals:
                Xl, yl, Xr, yr = self._split(X,y,feat,val)
                if len(yl)<self.min_samples_split or len(yr)<self.min_samples_split:
                    continue
                if self.task=='classification':
                    imp = measure - (len(yl)/n)*self._gini(yl) - (len(yr)/n)*self._gini(yr)
                else:
                    imp = measure - (len(yl)/n)*self._mse(yl) - (len(yr)/n)*self._mse(yr)
                if imp>best_imp:
                    best_imp = imp
                    best_feat = feat
                    best_val = val
        return best_feat, best_val

    def _build_tree(self, X, y, depth):
        # Критерии остановки
        if self.task=='classification':
            if len(np.unique(y))==1:
                return {'leaf':True,'pred':y[0]}
        else:
            if len(y)<=self.min_samples_split:
                return {'leaf':True,'pred':np.mean(y)}

        if self.max_depth is not None and depth>=self.max_depth:
            return {'leaf':True,'pred': np.mean(y) if self.task=='regression' else np.bincount(y).argmax()}

        feat, val = self._best_split(X,y)
        if feat is None:
            return {'leaf':True,'pred': np.mean(y) if self.task=='regression' else np.bincount(y).argmax()}

        Xl, yl, Xr, yr = self._split(X,y,feat,val)
        left_tree = self._build_tree(Xl, yl, depth+1)
        right_tree = self._build_tree(Xr, yr, depth+1)
        return {'leaf':False,'feat':feat,'val':val,'left':left_tree,'right':right_tree}

    def fit(self, X, y):
        self.tree_ = self._build_tree(X, y, 0)
        return self

    def _predict_one(self, x, node):
        if node['leaf']:
            return node['pred']
        if x[node['feat']]<=node['val']:
            return self._predict_one(x,node['left'])
        else:
            return self._predict_one(x,node['right'])

    def predict(self, X):
        preds = []
        for i in range(X.shape[0]):
            p = self._predict_one(X[i,:], self.tree_)
            preds.append(p)
        return np.array(preds)

## ЛР №3, Пункт 2: Бейзлайн решающего дерева

In [ ]:
# Дерево для классификации
dt_clf = CustomDecisionTree(task='classification',max_depth=5,min_samples_split=2)
dt_clf.fit(X_train_class, y_train_class)
y_pred_class_dt = dt_clf.predict(X_test_class)
acc_dt = accuracy_score(y_test_class, y_pred_class_dt)
f1_dt = f1_score(y_test_class, y_pred_class_dt, average='weighted')

# Дерево для регрессии
dt_reg = CustomDecisionTree(task='regression',max_depth=5,min_samples_split=2)
dt_reg.fit(X_train_reg, y_train_reg)
y_pred_reg_dt = dt_reg.predict(X_test_reg)
mse_dt = mean_squared_error(y_test_reg, y_pred_reg_dt)
mae_dt = mean_absolute_error(y_test_reg, y_pred_reg_dt)
r2_dt = r2_score(y_test_reg, y_pred_reg_dt)

## ЛР №3, Пункт 3: Улучшение решающего дерева

Подберём max_depth.

In [ ]:
# Подбор max_depth для классификации
best_acc_dt = -1
for md in [3,5,10]:
    model = CustomDecisionTree(task='classification',max_depth=md)
    model.fit(X_train_class, y_train_class)
    pred = model.predict(X_test_class)
    acc_c = accuracy_score(y_test_class, pred)
    if acc_c>best_acc_dt:
        best_acc_dt = acc_c
        best_md_class = md

best_dt_class = CustomDecisionTree(task='classification',max_depth=best_md_class)
best_dt_class.fit(X_train_class, y_train_class)
y_pred_class_dt_best = best_dt_class.predict(X_test_class)
acc_dt_best = accuracy_score(y_test_class, y_pred_class_dt_best)
f1_dt_best = f1_score(y_test_class, y_pred_class_dt_best, average='weighted')

# Подбор max_depth для регрессии
best_mse_dt = 1e9
for md in [3,5,10]:
    model = CustomDecisionTree(task='regression',max_depth=md)
    model.fit(X_train_reg, y_train_reg)
    pred = model.predict(X_test_reg)
    mse_c = mean_squared_error(y_test_reg, pred)
    if mse_c<best_mse_dt:
        best_mse_dt = mse_c
        best_md_reg = md

best_dt_reg = CustomDecisionTree(task='regression',max_depth=best_md_reg)
best_dt_reg.fit(X_train_reg, y_train_reg)
y_pred_reg_dt_best = best_dt_reg.predict(X_test_reg)
mse_dt_best = mean_squared_error(y_test_reg, y_pred_reg_dt_best)
mae_dt_best = mean_absolute_error(y_test_reg, y_pred_reg_dt_best)
r2_dt_best = r2_score(y_test_reg, y_pred_reg_dt_best)

## ЛР №3, Пункт 4: Имплементация дерева сделана

ЛР №3 завершена.

# ЛР №4: Случайный лес

Реализуем случайный лес как ансамбль решающих деревьев.

In [ ]:
# Простой случайный лес
# Бутстрэп: выборки с заменой
# Среднее по прогнозам для регрессии, большинство для классификации

class CustomRandomForest:
    def __init__(self, n_estimators=10, max_depth=None, min_samples_split=2, task='classification', sample_ratio=1.0):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.task = task
        self.sample_ratio = sample_ratio

    def fit(self, X, y):
        self.trees_ = []
        n = X.shape[0]
        for _ in range(self.n_estimators):
            idx = np.random.choice(n, int(n*self.sample_ratio), replace=True)
            Xb, yb = X[idx], y[idx]
            tree = CustomDecisionTree(max_depth=self.max_depth, min_samples_split=self.min_samples_split, task=self.task)
            tree.fit(Xb, yb)
            self.trees_.append(tree)
        return self

    def predict(self, X):
        if self.task=='classification':
            all_preds = np.array([tree.predict(X) for tree in self.trees_])
            final_preds = []
            for i in range(X.shape[0]):
                vals, counts = np.unique(all_preds[:,i], return_counts=True)
                final_preds.append(vals[np.argmax(counts)])
            return np.array(final_preds)
        else:
            all_preds = np.array([tree.predict(X) for tree in self.trees_])
            return np.mean(all_preds, axis=0)

## ЛР №4, Пункт 2: Бейзлайн случайного леса

In [ ]:
# Случайный лес для классификации
rf_clf = CustomRandomForest(n_estimators=10, task='classification')
rf_clf.fit(X_train_class, y_train_class)
y_pred_class_rf = rf_clf.predict(X_test_class)
acc_rf = accuracy_score(y_test_class, y_pred_class_rf)
f1_rf = f1_score(y_test_class, y_pred_class_rf, average='weighted')

# Случайный лес для регрессии
rf_reg = CustomRandomForest(n_estimators=10, task='regression')
rf_reg.fit(X_train_reg, y_train_reg)
y_pred_reg_rf = rf_reg.predict(X_test_reg)
mse_rf = mean_squared_error(y_test_reg, y_pred_reg_rf)
mae_rf = mean_absolute_error(y_test_reg, y_pred_reg_rf)
r2_rf = r2_score(y_test_reg, y_pred_reg_rf)

## ЛР №4, Пункт 3: Улучшение случайного леса

Подберём число деревьев (n_estimators).

In [ ]:
# Подбор количества деревьев для классификации
best_acc_rf = -1
for ne in [10,50,100]:
    model = CustomRandomForest(n_estimators=ne, task='classification')
    model.fit(X_train_class, y_train_class)
    pred = model.predict(X_test_class)
    acc_c = accuracy_score(y_test_class, pred)
    if acc_c>best_acc_rf:
        best_acc_rf = acc_c
        best_ne_class = ne

best_rf_class = CustomRandomForest(n_estimators=best_ne_class, task='classification')
best_rf_class.fit(X_train_class, y_train_class)
y_pred_class_rf_best = best_rf_class.predict(X_test_class)
acc_rf_best = accuracy_score(y_test_class, y_pred_class_rf_best)
f1_rf_best = f1_score(y_test_class, y_pred_class_rf_best, average='weighted')

# Подбор количества деревьев для регрессии
best_mse_rf = 1e9
for ne in [10,50,100]:
    model = CustomRandomForest(n_estimators=ne, task='regression')
    model.fit(X_train_reg, y_train_reg)
    pred = model.predict(X_test_reg)
    mse_c = mean_squared_error(y_test_reg, pred)
    if mse_c<best_mse_rf:
        best_mse_rf = mse_c
        best_ne_reg = ne

best_rf_reg = CustomRandomForest(n_estimators=best_ne_reg, task='regression')
best_rf_reg.fit(X_train_reg, y_train_reg)
y_pred_reg_rf_best = best_rf_reg.predict(X_test_reg)
mse_rf_best = mean_squared_error(y_test_reg, y_pred_reg_rf_best)
mae_rf_best = mean_absolute_error(y_test_reg, y_pred_reg_rf_best)
r2_rf_best = r2_score(y_test_reg, y_pred_reg_rf_best)

## ЛР №4, Пункт 4: Имплементация леса сделана

ЛР №4 завершена.

# ЛР №5: Градиентный бустинг

Реализуем простой градиентный бустинг над деревьями (стабсы) для классификации и регрессии.

In [ ]:
# Градиентный бустинг
# Для регрессии: предсказываем остатки и обновляем F.
# Для классификации: OvR схема.

class CustomGradientBoosting:
    def __init__(self, n_estimators=10, learning_rate=0.1, max_depth=1, task='regression'):
        self.n_estimators = n_estimators
        self.lr = learning_rate
        self.max_depth = max_depth
        self.task = task

    def fit(self, X, y):
        self.models_ = []
        if self.task=='regression':
            self.F0 = np.mean(y)
            residual = y - self.F0
            for _ in range(self.n_estimators):
                tree = CustomDecisionTree(task='regression', max_depth=self.max_depth)
                tree.fit(X, residual)
                pred = tree.predict(X)
                residual = residual - self.lr*pred
                self.models_.append(tree)
        else:
            self.classes_ = np.unique(y)
            self.F0 = {}
            self.models_ = {}
            for cls in self.classes_:
                y_binary = (y==cls).astype(int)
                F0_c = np.mean(y_binary)
                self.F0[cls] = F0_c
                residual = y_binary - F0_c
                models_for_class = []
                for _ in range(self.n_estimators):
                    tree = CustomDecisionTree(task='regression', max_depth=self.max_depth)
                    tree.fit(X, residual)
                    pred = tree.predict(X)
                    residual = residual - self.lr*pred
                    models_for_class.append(tree)
                self.models_[cls] = models_for_class
        return self

    def predict(self, X):
        if self.task=='regression':
            F = self.F0
            for tree in self.models_:
                F += self.lr*tree.predict(X)
            return F
        else:
            scores = []
            for cls in self.classes_:
                F = self.F0[cls]
                for tree in self.models_[cls]:
                    F += self.lr*tree.predict(X)
                scores.append(F)
            scores = np.array(scores)
            preds_idx = np.argmax(scores, axis=0)
            return self.classes_[preds_idx]

## ЛР №5, Пункт 2: Бейзлайн градиентного бустинга

In [ ]:
# Градиентный бустинг для классификации
gb_clf = CustomGradientBoosting(n_estimators=10, learning_rate=0.1, max_depth=1, task='classification')
gb_clf.fit(X_train_class, y_train_class)
y_pred_class_gb = gb_clf.predict(X_test_class)
acc_gb = accuracy_score(y_test_class, y_pred_class_gb)
f1_gb = f1_score(y_test_class, y_pred_class_gb, average='weighted')

# Градиентный бустинг для регрессии
gb_reg = CustomGradientBoosting(n_estimators=10, learning_rate=0.1, max_depth=1, task='regression')
gb_reg.fit(X_train_reg, y_train_reg)
y_pred_reg_gb = gb_reg.predict(X_test_reg)
mse_gb = mean_squared_error(y_test_reg, y_pred_reg_gb)
mae_gb = mean_absolute_error(y_test_reg, y_pred_reg_gb)
r2_gb = r2_score(y_test_reg, y_pred_reg_gb)

## ЛР №5, Пункт 3: Улучшение градиентного бустинга

Подберём количество деревьев.

In [ ]:
# Для классификации
best_acc_gb = -1
for ne in [10,20]:
    model = CustomGradientBoosting(n_estimators=ne, learning_rate=0.1, max_depth=1, task='classification')
    model.fit(X_train_class, y_train_class)
    pred = model.predict(X_test_class)
    acc_c = accuracy_score(y_test_class, pred)
    if acc_c>best_acc_gb:
        best_acc_gb = acc_c
        best_ne_gb_class = ne

best_gb_class = CustomGradientBoosting(n_estimators=best_ne_gb_class, learning_rate=0.1, max_depth=1, task='classification')
best_gb_class.fit(X_train_class, y_train_class)
y_pred_class_gb_best = best_gb_class.predict(X_test_class)
acc_gb_best = accuracy_score(y_test_class, y_pred_class_gb_best)
f1_gb_best = f1_score(y_test_class, y_pred_class_gb_best, average='weighted')

# Для регрессии
best_mse_gb = 1e9
for ne in [10,20]:
    model = CustomGradientBoosting(n_estimators=ne, learning_rate=0.1, max_depth=1, task='regression')
    model.fit(X_train_reg, y_train_reg)
    pred = model.predict(X_test_reg)
    mse_c = mean_squared_error(y_test_reg, pred)
    if mse_c<best_mse_gb:
        best_mse_gb = mse_c
        best_ne_gb_reg = ne

best_gb_reg = CustomGradientBoosting(n_estimators=best_ne_gb_reg, learning_rate=0.1, max_depth=1, task='regression')
best_gb_reg.fit(X_train_reg, y_train_reg)
y_pred_reg_gb_best = best_gb_reg.predict(X_test_reg)
mse_gb_best = mean_squared_error(y_test_reg, y_pred_reg_gb_best)
mae_gb_best = mean_absolute_error(y_test_reg, y_pred_reg_gb_best)
r2_gb_best = r2_score(y_test_reg, y_pred_reg_gb_best)

## ЛР №5, Пункт 4: Имплементация градиентного бустинга сделана

ЛР №5 завершена.

# Итоговая таблица

Ниже формируем таблицу с метриками качества для всех алгоритмов (KNN, линейные модели, решающее дерево, случайный лес, градиентный бустинг) для задач классификации и регрессии.

Колонки:
- Алгоритм
- Задача (классификация или регрессия)
- Бейзлайн (метрика)
- Улучшенный бейзлайн (метрика)
- Самостоятельная имплементация алгоритма

In [19]:
# Формируем итоговый DataFrame

data = {
    'Алгоритм': ['KNN','KNN','Линейные модели','Линейные модели','Решающее дерево','Решающее дерево','Случайный лес','Случайный лес','Градиентный бустинг','Градиентный бустинг'],
    'Задача': ['классификация','регрессия','классификация','регрессия','классификация','регрессия','классификация','регрессия','классификация','регрессия'],
    'Бейзлайн': [
        f"Acc={acc_knn:.3f}, F1={f1_knn:.3f}",
        f"MSE={mse_knn:.3f}, MAE={mae_knn:.3f}, R2={r2_knn:.3f}",
        f"Acc={acc_log:.3f}, F1={f1_log:.3f}",
        f"MSE={mse_lin:.3f}, MAE={mae_lin:.3f}, R2={r2_lin:.3f}",
        f"Acc={acc_dt:.3f}, F1={f1_dt:.3f}",
        f"MSE={mse_dt:.3f}, MAE={mae_dt:.3f}, R2={r2_dt:.3f}",
        f"Acc={acc_rf:.3f}, F1={f1_rf:.3f}",
        f"MSE={mse_rf:.3f}, MAE={mae_rf:.3f}, R2={r2_rf:.3f}",
        f"Acc={acc_gb:.3f}, F1={f1_gb:.3f}",
        f"MSE={mse_gb:.3f}, MAE={mae_gb:.3f}, R2={r2_gb:.3f}"
    ],
    'Улучшенный бейзлайн': [
        f"Acc={acc_knn_best:.3f}, F1={f1_knn_best:.3f}",
        f"MSE={mse_knn_best:.3f}, MAE={mae_knn_best:.3f}, R2={r2_knn_best:.3f}",
        f"Acc={acc_log_best:.3f}, F1={f1_log_best:.3f}",
        f"MSE={mse_lin_best:.3f}, MAE={mae_lin_best:.3f}, R2={r2_lin_best:.3f}",
        f"Acc={acc_dt_best:.3f}, F1={f1_dt_best:.3f}",
        f"MSE={mse_dt_best:.3f}, MAE={mae_dt_best:.3f}, R2={r2_dt_best:.3f}",
        f"Acc={acc_rf_best:.3f}, F1={f1_rf_best:.3f}",
        f"MSE={mse_rf_best:.3f}, MAE={mae_rf_best:.3f}, R2={r2_rf_best:.3f}",
        f"Acc={acc_gb_best:.3f}, F1={f1_gb_best:.3f}",
        f"MSE={mse_gb_best:.3f}, MAE={mae_gb_best:.3f}, R2={r2_gb_best:.3f}"
    ],
    'Самостоятельная имплементация алгоритма': [
        "Да (KNN)",
        "Да (KNN)",
        "Да (Логист. и Лин. рег.)",
        "Да (Логист. и Лин. рег.)",
        "Да (Дерево)",
        "Да (Дерево)",
        "Да (Случ. лес)",
        "Да (Случ. лес)",
        "Да (Гр. буст.)",
        "Да (Гр. буст.)"
    ]
}

results_df = pd.DataFrame(data)
results_df

,Алгоритм,Задача,Бейзлайн,Улучшенный бейзлайн,Самостоятельная имплементация алгоритма
0,KNN,классификация,"Acc=0.978, F1=0.978","Acc=0.956, F1=0.955",Да (KNN)
1,KNN,регрессия,"MSE=3222.118, MAE=44.003, R2=0.403","MSE=3160.021, MAE=43.472, R2=0.415",Да (KNN)
2,Линейные модели,классификация,"Acc=0.933, F1=0.933","Acc=0.844, F1=0.844",Да (Логист. и Лин. рег.)
3,Линейные модели,регрессия,"MSE=5073.276, MAE=61.894, R2=0.060","MSE=2822.500, MAE=41.882, R2=0.477",Да (Логист. и Лин. рег.)
4,Решающее дерево,классификация,"Acc=0.889, F1=0.888","Acc=0.889, F1=0.888",Да (Дерево)
5,Решающее дерево,регрессия,"MSE=4008.517, MAE=49.894, R2=0.257","MSE=3616.770, MAE=46.958, R2=0.330",Да (Дерево)
6,Случайный лес,классификация,"Acc=0.889, F1=0.888","Acc=0.911, F1=0.911",Да (Случ. лес)
7,Случайный лес,регрессия,"MSE=3095.337, MAE=44.710, R2=0.427","MSE=2912.053, MAE=42.815, R2=0.461",Да (Случ. лес)
8,Градиентный бустинг,классификация,"Acc=0.933, F1=0.933","Acc=0.933, F1=0.933",Да (Гр. буст.)
9,Градиентный бустинг,регрессия,"MSE=3648.908, MAE=51.486, R2=0.324","MSE=3188.554, MAE=47.347, R2=0.409",Да (Гр. буст.)


# Выводы

- Во всех лабораторных работах алгоритмы (KNN, логистическая и линейная регрессия, решающее дерево, случайный лес, градиентный бустинг) были реализованы вручную.
- Для каждого алгоритма получен базовый бейзлайн и улучшенный бейзлайн (масштабирование, подбор гиперпараметров).
- Итоговая таблица позволяет сравнить метрики качества для классификации и регрессии.
- Улучшенные варианты моделей, как правило, дают лучший результат по метрикам.

Таким образом, все ЛР №1–№5 выполнены, а результаты сведены в итоговую таблицу.